In [1]:
%matplotlib inline

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

# # Encoder NN
start_traj = 192
start_dis = 528
start_end = 2350

index_encoder0 = np.arange(0, start_traj)
dim_encoder0 = [512]
activation_encoder0 = [0]

index_encoder1 = np.arange(start_traj, start_dis)
dim_encoder1 = [512]
activation_encoder1 = [0]

index_encoder2 = np.arange(start_dis, start_end)
dim_encoder2 = [512]
activation_encoder2 = [0]


index_encoders = [index_encoder0, index_encoder1, index_encoder2]
dim_encoders = [dim_encoder0, dim_encoder1, dim_encoder2]
activation_encoders = [activation_encoder0, activation_encoder1, activation_encoder2]

# Keep prob
keep_prob_main = 0.7
keep_prob_encoders = [0.7, 0.7, 0.7]



# Tuning para
learning_rate = 0.0001

def main():
    GPU_Occupancy = 0.9
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_Occupancy)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

    mann = MainNN()
    mann.BuildModel(load_path, save_path,
                    type_normalization,
                    hiddenDim=hiddenDim,
                    activations=activations,

                    index_encoders=index_encoders,
                    dim_encoders=dim_encoders,
                    activation_encoders=activation_encoders)
    mann.Train(sess,
               # Model name for Saving
               name_model,
               # Flag/Path of test data
               path_test=test_path,
               flag_save_bin = True,
               step_save=100, max_save=3,
               # HyperParameters
               keep_prob_main=keep_prob_main, batch_size=32, epoch=500, learning_rate_ini=learning_rate,
               keep_prob_encoders=keep_prob_encoders
               )


In [2]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.pose_fc = nn.Linear(198, 512)
        self.traj_fc = nn.Linear(336, 512)
        self.sensor_fc = nn.Linear(1822, 512)
        self.res_fc = nn.Linear(1536, 1536)
        self.decoder_fc = nn.Linear(1536, 220)
        # まとめたいけど、わからん
        self.Residual_Dence_Block = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1536, 1536),
        )

    def forward(self, pose, traj, sensor):
        x = self.flatten(x)
        pose = self.pose_fc(pose)
        traj = self.traj_fc(traj)
        sensor = self.sensor_fc(sensor)
        H_zero = torch.cat(pose, traj, sensor, dim=1)
        H_zero  = nn.ReLU(H_zero)
        H_one = self.res_fc(H_zero) + H_zero
        H_one  = nn.ReLU(H_one)
        H_two = self.res_fc(H_one) + H_one + H_zero
        H_two  = nn.ReLU(H_two)
        H_three = self.res_fc(H_two) + H_two
        H_three  = nn.ReLU(H_three)
        H_four = self.res_fc(H_three) + H_three + H_two
        out = self.decoder_fc(H_four)
        return out

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (pose_fc): Linear(in_features=198, out_features=512, bias=True)
  (traj_fc): Linear(in_features=336, out_features=512, bias=True)
  (sensor_fc): Linear(in_features=1822, out_features=512, bias=True)
  (res_fc): Linear(in_features=1536, out_features=1536, bias=True)
  (decoder_fc): Linear(in_features=1536, out_features=220, bias=True)
  (Residual_Dence_Block): Sequential(
    (0): ReLU()
    (1): Linear(in_features=1536, out_features=1536, bias=True)
  )
)


In [ ]:
print()
# net.state_dict()['conv1.weight'][0] = torch.tensor([conv1の新しいパラメータ])
# print(net.state_dict()['conv1.weight'])